# Chain of Responsibility Pattern

## Intent
Avoid coupling the sender of a request to its receiver by giving more than one object a chance to handle the request. Chain the receiving objects and pass the request along the chain until an object handles it.

## Problem
You need to process a request but:
- Multiple objects can handle it
- Handler isn't known in advance
- Don't want to hardcode handler references
- Want to issue request without knowing exact handler

**Real-world examples:**
- Support ticket escalation
- Logging systems
- Middleware pipelines
- Approval workflows

## When to Use
✅ **Use when:**
- Multiple objects can handle request
- Handler set isn't known in advance
- Want to decouple sender from receiver
- Want to add handlers dynamically

❌ **Avoid when:**
- Only one handler exists
- Every request must be handled
- Chain order doesn't matter (use composite instead)

## Pattern Structure
```
┌────────┐        ┌─────────┐
│ Client │───────►│ Handler │
└────────┘        ├─────────┤
                  │ next    │
                  │handle() │
                  └─────────┘
                       ▲
              ┌────────┴────────┐
         ┌────┴───┐      ┌─────┴─────┐
         │Handler1│      │Handler2   │
         ├────────┤      ├───────────┤
         │handle()│      │handle()   │
         └────────┘      └───────────┘
```

## Example 1: Support Ticket System (Without Chain)

**Problem**: Hardcoded escalation logic

In [ ]:
# WITHOUT Chain of Responsibility
class SupportSystem:
    def handle_ticket(self, priority, message):
        # Hardcoded escalation logic
        if priority == "low":
            print(f"Level 1 Support: Handling '{message}'")
        elif priority == "medium":
            print(f"Level 2 Support: Handling '{message}'")
        elif priority == "high":
            print(f"Manager: Handling '{message}'")
        elif priority == "critical":
            print(f"Director: Handling '{message}'")
        else:
            print(f"Unknown priority: {priority}")
        
        # Adding new levels requires modifying this code!

# Usage
print("=== Without Chain of Responsibility ===")
support = SupportSystem()
support.handle_ticket("low", "Password reset")
support.handle_ticket("high", "Server down")

print("\n❌ Tightly coupled! Hard to extend!")

## Implementation: Chain of Responsibility Pattern

In [ ]:
from abc import ABC, abstractmethod
from typing import Optional

# Handler interface
class SupportHandler(ABC):
    """Abstract handler in the chain."""
    
    def __init__(self):
        self._next_handler: Optional[SupportHandler] = None
    
    def set_next(self, handler: 'SupportHandler') -> 'SupportHandler':
        """Set the next handler in the chain."""
        self._next_handler = handler
        return handler  # Allow chaining: h1.set_next(h2).set_next(h3)
    
    @abstractmethod
    def handle(self, priority: str, message: str) -> bool:
        """Handle request or pass to next handler."""
        pass
    
    def _pass_to_next(self, priority: str, message: str) -> bool:
        """Pass request to next handler."""
        if self._next_handler:
            return self._next_handler.handle(priority, message)
        return False  # No handler could process


# Concrete handlers
class Level1Support(SupportHandler):
    """Handles low priority tickets."""
    
    def handle(self, priority: str, message: str) -> bool:
        if priority == "low":
            print(f"  👨‍💻 Level 1 Support: Resolved '{message}'")
            return True
        else:
            print(f"  Level 1: Cannot handle '{priority}' priority. Escalating...")
            return self._pass_to_next(priority, message)


class Level2Support(SupportHandler):
    """Handles medium priority tickets."""
    
    def handle(self, priority: str, message: str) -> bool:
        if priority == "medium":
            print(f"  👨‍💼 Level 2 Support: Resolved '{message}'")
            return True
        else:
            print(f"  Level 2: Cannot handle '{priority}' priority. Escalating...")
            return self._pass_to_next(priority, message)


class ManagerSupport(SupportHandler):
    """Handles high priority tickets."""
    
    def handle(self, priority: str, message: str) -> bool:
        if priority == "high":
            print(f"  👔 Manager: Resolved '{message}'")
            return True
        else:
            print(f"  Manager: Cannot handle '{priority}' priority. Escalating...")
            return self._pass_to_next(priority, message)


class DirectorSupport(SupportHandler):
    """Handles critical priority tickets."""
    
    def handle(self, priority: str, message: str) -> bool:
        if priority == "critical":
            print(f"  🎩 Director: Resolved '{message}'")
            return True
        else:
            print(f"  Director: Cannot handle '{priority}' priority.")
            return self._pass_to_next(priority, message)


# Demo
print("\n=== Chain of Responsibility Pattern ===")

# Build the chain
level1 = Level1Support()
level2 = Level2Support()
manager = ManagerSupport()
director = DirectorSupport()

# Link handlers: Level1 → Level2 → Manager → Director
level1.set_next(level2).set_next(manager).set_next(director)

# Process tickets
tickets = [
    ("low", "Password reset"),
    ("medium", "Software installation"),
    ("high", "Server down"),
    ("critical", "Data breach")
]

for priority, message in tickets:
    print(f"\n🎫 Ticket: [{priority.upper()}] {message}")
    handled = level1.handle(priority, message)
    if not handled:
        print("  ❌ Ticket could not be handled!")

print("\n✅ Chain handles requests dynamically!")

## Real-World Example: Logging System

In [ ]:
from enum import IntEnum
from datetime import datetime

# Log levels
class LogLevel(IntEnum):
    DEBUG = 1
    INFO = 2
    WARNING = 3
    ERROR = 4
    CRITICAL = 5


# Handler interface
class Logger(ABC):
    """Abstract logger in the chain."""
    
    def __init__(self, level: LogLevel):
        self.level = level
        self._next_logger: Optional[Logger] = None
    
    def set_next(self, logger: 'Logger') -> 'Logger':
        self._next_logger = logger
        return logger
    
    def log(self, level: LogLevel, message: str) -> None:
        """Log message if level is appropriate."""
        if level >= self.level:
            self._write(message)
        
        # Always pass to next logger
        if self._next_logger:
            self._next_logger.log(level, message)
    
    @abstractmethod
    def _write(self, message: str) -> None:
        pass


# Concrete loggers
class ConsoleLogger(Logger):
    """Logs to console."""
    
    def _write(self, message: str) -> None:
        timestamp = datetime.now().strftime("%H:%M:%S")
        print(f"  🖥️  Console [{timestamp}]: {message}")


class FileLogger(Logger):
    """Logs to file."""
    
    def __init__(self, level: LogLevel, filename: str):
        super().__init__(level)
        self.filename = filename
    
    def _write(self, message: str) -> None:
        timestamp = datetime.now().strftime("%H:%M:%S")
        print(f"  📄 File [{timestamp}] → {self.filename}: {message}")


class EmailLogger(Logger):
    """Logs to email (for critical issues)."""
    
    def __init__(self, level: LogLevel, recipient: str):
        super().__init__(level)
        self.recipient = recipient
    
    def _write(self, message: str) -> None:
        timestamp = datetime.now().strftime("%H:%M:%S")
        print(f"  📧 Email [{timestamp}] → {self.recipient}: {message}")


# Demo
print("\n=== Logging Chain ===")

# Build chain: Console (INFO+) → File (WARNING+) → Email (ERROR+)
console = ConsoleLogger(LogLevel.INFO)
file = FileLogger(LogLevel.WARNING, "app.log")
email = EmailLogger(LogLevel.ERROR, "admin@example.com")

console.set_next(file).set_next(email)

# Log messages at different levels
print("\n1. DEBUG message (nobody logs):")
console.log(LogLevel.DEBUG, "Variable x = 5")

print("\n2. INFO message (console only):")
console.log(LogLevel.INFO, "User logged in")

print("\n3. WARNING message (console + file):")
console.log(LogLevel.WARNING, "Disk space low")

print("\n4. ERROR message (all loggers):")
console.log(LogLevel.ERROR, "Database connection failed")

print("\n5. CRITICAL message (all loggers):")
console.log(LogLevel.CRITICAL, "System crash imminent!")

print("\n✅ Multiple handlers process same request!")

## Real-World Example: Expense Approval Chain

In [ ]:
# Handler interface
class Approver(ABC):
    """Abstract expense approver."""
    
    def __init__(self, name: str, limit: float):
        self.name = name
        self.limit = limit
        self._next_approver: Optional[Approver] = None
    
    def set_next(self, approver: 'Approver') -> 'Approver':
        self._next_approver = approver
        return approver
    
    def approve(self, amount: float, purpose: str) -> bool:
        """Approve expense or escalate."""
        if amount <= self.limit:
            print(f"  ✅ {self.name}: Approved ${amount:.2f} for '{purpose}'")
            return True
        elif self._next_approver:
            print(f"  ⬆️  {self.name}: Amount exceeds limit (${self.limit:.2f}). Escalating...")
            return self._next_approver.approve(amount, purpose)
        else:
            print(f"  ❌ {self.name}: Amount ${amount:.2f} exceeds all limits. REJECTED.")
            return False


# Concrete approvers
class TeamLead(Approver):
    def __init__(self, name: str):
        super().__init__(name, limit=1000.0)


class Manager(Approver):
    def __init__(self, name: str):
        super().__init__(name, limit=5000.0)


class Director(Approver):
    def __init__(self, name: str):
        super().__init__(name, limit=20000.0)


class CEO(Approver):
    def __init__(self, name: str):
        super().__init__(name, limit=100000.0)


# Demo
print("\n=== Expense Approval Chain ===")

# Build approval chain
alice = TeamLead("Alice (Team Lead)")
bob = Manager("Bob (Manager)")
charlie = Director("Charlie (Director)")
diana = CEO("Diana (CEO)")

alice.set_next(bob).set_next(charlie).set_next(diana)

# Submit expense requests
expenses = [
    (500, "Office supplies"),
    (2500, "New laptop"),
    (15000, "Conference sponsorship"),
    (75000, "New server infrastructure"),
    (150000, "Company acquisition")
]

for amount, purpose in expenses:
    print(f"\n💰 Expense Request: ${amount:.2f} - {purpose}")
    alice.approve(amount, purpose)

print("\n✅ Chain automatically escalates to appropriate level!")

## Real-World Example: Authentication Middleware

In [ ]:
from typing import Dict

# Request object
class Request:
    def __init__(self, user: str, password: str, role: str = None):
        self.user = user
        self.password = password
        self.role = role
        self.authenticated = False
        self.authorized = False


# Middleware handler
class Middleware(ABC):
    """Abstract middleware."""
    
    def __init__(self):
        self._next: Optional[Middleware] = None
    
    def set_next(self, middleware: 'Middleware') -> 'Middleware':
        self._next = middleware
        return middleware
    
    def handle(self, request: Request) -> bool:
        """Process request and pass to next middleware."""
        if self._check(request):
            if self._next:
                return self._next.handle(request)
            return True  # End of chain, success!
        return False
    
    @abstractmethod
    def _check(self, request: Request) -> bool:
        pass


# Concrete middleware
class AuthenticationMiddleware(Middleware):
    """Checks username and password."""
    
    def __init__(self):
        super().__init__()
        self._users: Dict[str, str] = {
            "alice": "pass123",
            "bob": "secret456"
        }
    
    def _check(self, request: Request) -> bool:
        print(f"  🔐 Authentication: Checking credentials for '{request.user}'...")
        
        if request.user in self._users and self._users[request.user] == request.password:
            request.authenticated = True
            print(f"  ✅ Authentication: Success")
            return True
        else:
            print(f"  ❌ Authentication: Failed")
            return False


class RoleCheckMiddleware(Middleware):
    """Checks user role."""
    
    def __init__(self):
        super().__init__()
        self._roles: Dict[str, str] = {
            "alice": "admin",
            "bob": "user"
        }
    
    def _check(self, request: Request) -> bool:
        print(f"  👤 Role Check: Verifying role...")
        
        if request.user in self._roles:
            request.role = self._roles[request.user]
            print(f"  ✅ Role Check: User is '{request.role}'")
            return True
        else:
            print(f"  ❌ Role Check: No role found")
            return False


class ThrottlingMiddleware(Middleware):
    """Checks request rate."""
    
    def __init__(self, max_requests: int):
        super().__init__()
        self.max_requests = max_requests
        self._request_count: Dict[str, int] = {}
    
    def _check(self, request: Request) -> bool:
        print(f"  ⏱️  Throttling: Checking request rate...")
        
        count = self._request_count.get(request.user, 0)
        
        if count < self.max_requests:
            self._request_count[request.user] = count + 1
            print(f"  ✅ Throttling: OK ({count + 1}/{self.max_requests})")
            return True
        else:
            print(f"  ❌ Throttling: Rate limit exceeded")
            return False


# Demo
print("\n=== Authentication Middleware Chain ===")

# Build middleware chain
auth = AuthenticationMiddleware()
role = RoleCheckMiddleware()
throttle = ThrottlingMiddleware(max_requests=3)

auth.set_next(role).set_next(throttle)

# Test requests
print("\n1. Valid request:")
req1 = Request("alice", "pass123")
if auth.handle(req1):
    print(f"  🎉 Request processed! Role: {req1.role}")

print("\n2. Invalid password:")
req2 = Request("alice", "wrong")
if not auth.handle(req2):
    print(f"  🚫 Request rejected")

print("\n3. Multiple requests (testing throttling):")
for i in range(4):
    print(f"\n  Request #{i+1}:")
    req = Request("bob", "secret456")
    result = auth.handle(req)
    if result:
        print(f"    ✅ Passed")
    else:
        print(f"    ❌ Blocked")

print("\n✅ Middleware chain processes requests sequentially!")

## Advantages & Disadvantages

### ✅ Advantages
1. **Decoupling**: Sender doesn't know receiver
2. **Flexibility**: Add/remove handlers dynamically
3. **Single Responsibility**: Each handler has one job
4. **Order control**: Control processing order
5. **Open/Closed Principle**: Add handlers without modifying existing code

### ❌ Disadvantages
1. **No guarantee**: Request may not be handled
2. **Hard to debug**: Request path not obvious
3. **Performance**: May traverse entire chain
4. **Complex chains**: Long chains are hard to understand

## When to Stop Processing?

**Stop after first handler** (like support ticket):
```python
if can_handle:
    handle_it()
    return True  # Stop here
else:
    pass_to_next()
```

**Process all handlers** (like logging):
```python
if can_handle:
    handle_it()
pass_to_next()  # Always continue
```

## Common Use Cases

1. **Support systems**: Ticket escalation
2. **Logging**: Multiple log destinations
3. **Middleware**: Request processing pipelines
4. **Authorization**: Multi-level approval
5. **Event handling**: GUI event propagation
6. **Error handling**: Exception handling chain
7. **Validation**: Multi-step validation

## Related Patterns

- **Command**: Often uses chain to determine handler
- **Composite**: Chain is linear, Composite is tree
- **Decorator**: Chain changes behavior, Decorator adds behavior

## Best Practices

1. **Default handler**: Add catch-all at end of chain
2. **Clear responsibilities**: Each handler should have clear purpose
3. **Logging**: Log when passing to next handler
4. **Avoid cycles**: Don't create circular chains
5. **Keep handlers simple**: Single responsibility
6. **Document order**: Chain order matters!

## Summary

Chain of Responsibility enables:
- Decoupled request handling
- Dynamic handler configuration
- Multiple handlers for same request
- Flexible processing order

Perfect for: Support systems, logging, middleware, approvals, event handling.

**Key Insight**: Pass requests along a chain of handlers, allowing multiple objects a chance to handle it without coupling sender to receiver!